In [92]:
import json
import os
import sys
import uuid
from dotenv import load_dotenv
from azure.core.exceptions import AzureError
from azure.cosmos import CosmosClient, PartitionKey, exceptions

In [93]:
env_path = ".env"
load_dotenv(env_path)

True

In [94]:
ENDPOINT = os.environ["COSMOS_ENDPOINT"]
KEY = os.environ["COSMOS_KEY"]

client = CosmosClient(url=ENDPOINT, credential=KEY)

In [95]:
DATABASE = "transcriptions-db"
CONTAINER_TRANSCRIPTION = 'container-result-transcription'
CONTAINER_METADADOS = 'metadata-container'
CONTAINER_LOGS = 'logs-container'

In [96]:
database = client.get_database_client(DATABASE)
container_transcription = database.get_container_client(CONTAINER_TRANSCRIPTION)

In [97]:
def inserir_transcricoes(container, data, filename):
    try:
        transcricao_item = {
            'id': str(uuid.uuid4()),
            'prompt': data['prompt'],
            'modelo': data['model'],
            'audio_name': filename,
            'audio_path': data['audio_path'],
            'transcription': data['transcription']
        }
        container.create_item(body=transcricao_item)
        print("Transcrições inseridas com sucesso!")
    except exceptions.CosmosHttpResponseError as e:
        print(f"Erro ao inserir transcrições: {e}")

In [98]:
def upload_files_to_blob_and_insert_to_cosmos(container_client, PATH):
    for filename in os.listdir(PATH):
        print("Verificando o arquivo ", filename)
        file_path = f'{PATH}/{filename}'
    
        if filename.endswith('.json'):
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
            try:
                if 'transcription' in data:
                    inserir_transcricoes(container_client, data, filename)
            except Exception as e:
                print(f"Erro ao inserir dados no CosmosDB: {e}")

In [99]:
upload_files_to_blob_and_insert_to_cosmos(container_transcription, './json_files')

Verificando o arquivo  2874774.json
Transcrições inseridas com sucesso!
Métricas inseridas com sucesso!
Verificando o arquivo  2874830.json
Transcrições inseridas com sucesso!
Métricas inseridas com sucesso!
Verificando o arquivo  2961972.json
Transcrições inseridas com sucesso!
Métricas inseridas com sucesso!
Verificando o arquivo  2962046.json
Transcrições inseridas com sucesso!
Métricas inseridas com sucesso!
Verificando o arquivo  2962074.json
Transcrições inseridas com sucesso!
Métricas inseridas com sucesso!
Verificando o arquivo  2962184.json
Transcrições inseridas com sucesso!
Métricas inseridas com sucesso!
Verificando o arquivo  2962219.json
Transcrições inseridas com sucesso!
Métricas inseridas com sucesso!
Verificando o arquivo  2962307.json
Transcrições inseridas com sucesso!
Métricas inseridas com sucesso!
Verificando o arquivo  2962417.json
Transcrições inseridas com sucesso!
Métricas inseridas com sucesso!
Verificando o arquivo  2962505.json
Transcrições inseridas com s